In [3]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
I0316 15:59:07.521984 140380530575168 file_utils.py:38] PyTorch version 1.4.0 available.
I0316 15:59:09.284233 140380530575168 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
Loading faiss with AVX2 support.
I0316 15:59:09.469179 140380530575168 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [32]:
config = Config()
task = Task(config)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 3.55 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0346 seconds


RuntimeError: CUDA error: out of memory

In [30]:
toy_data = load_data("TOY", "spacy")
vocab = toy_data['vocab']

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 4.25 seconds


In [9]:
import torch
nbr_tensor = torch.randint(10000, size=(10000, 10))
embed = torch.rand(10000, 300)

In [5]:
src_tokens = torch.randint(10000, size=(4, 16))

In [10]:
nbrs = nbr_tensor[src_tokens]
embed[nbrs].size()

torch.Size([4, 16, 10, 300])

In [24]:
from allennlpx.modules.graph_embedding import GraphEmbedding
embedder = GraphEmbedding(embedding_dim=300, 
                          num_embeddings=10000,
                          neighbours=torch.randint(10000, size=(10000, 10)))
embedder(src_tokens).size()

torch.Size([4, 16, 300])

In [46]:
from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher
# def build_neighbour_indices(searcher: EmbeddingSearcher):
#     stacks = []
#     for i in range(searcher.embed.size(0)):
#         _, I = searcher.search(i)
#         print(I)
#         if i == 10:
#             break
        
# searcher = EmbeddingSearcher(embed=torch.rand(vocab.get_vocab_size(), 100),
#                   word2idx=vocab.get_token_index, 
#                   idx2word=vocab.get_token_from_index)
# build_neighbour_indices(searcher)

import faiss
index = faiss.IndexFlatL2(100)
res = faiss.StandardGpuResources()  # use a single GPU
index = faiss.index_cpu_to_gpu(res, 0, index)
embed = torch.rand(vocab.get_vocab_size(), 100).numpy()
index.add(embed)
_, I = index.search(embed, k=10)